## Import Libraries

In [2]:
from os import path, makedirs
from itertools import product
import numpy as np
import scipy.io as sio

from brainpipe.system import study
from brainpipe.statistics import *
from utils import rename_elecs

## Power Decoding - Partial//Detailed Encoding
### For ALL time points

In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

st = study('Olfacto')
exp, meth = 'Enc', 'BETWEEN'#'Enc'
conds, subjects = ['early','late'],['PIRJ','FERJ','CHAF','LEFC','VACJ','SEMC']
freqs = ['theta']
#path_pow = path.join(st.path, 'feature/TPSim_2groups_Enc/TPS_btw_v=1_elecs=all/')
path_pow = path.join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')

filename = path.join(path_pow, 'TPS_pears_learn_{}_btw_{}_2gr.npz') #su cond freq
save_path = path.join(st.path, 'classified/TPSim_clf_EarlyLate_theta_last_analyses/')
#su, method(global,btw,wth),exp (E,R,ER),freq
if exp == 'Enc_Ret':
    name_classif = path.join(save_path, 
            '{}_LDA_clf_'+meth+'_E_R_'+conds[0]+'_'+conds[1]+'_{}_k=5.npz')
else:
    name_classif = path.join(save_path, 
         '{}_LDA_clf_'+meth+'_'+exp[0]+'_'+conds[0]+'_'+conds[1]+'_{}_k=5.npz')
rois_to_keep = ['ACC','aHC','IFG','Ins_olf','MFG','OFC_olf','PHG','SFG','pPirT','Amg']

nperm = 1000

if not path.exists(save_path):
    makedirs(save_path)

for su,freq in product(subjects,freqs):
    #for freq in freqs:
    pow_list = []
    #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
    mat0 = np.load(filename.format(su,freq),allow_pickle=True)
    labels = mat0['label']
    x,y,z = mat0['xyz'][:,0], mat0['xyz'][:,1], mat0['xyz'][:,2]
    new_labels = rename_elecs(labels,x,y,z)
    idx_rois = np.isin(new_labels, rois_to_keep)
    
    pow_list.append(mat0['tps_0'][idx_rois])
    nelecs = mat0['tps_0'][idx_rois].shape[0]
    mat1 = np.load(filename.format(su,freq),allow_pickle=True)
    print(mat1.files)
    pow_list.append(mat1['tps_1'][idx_rois])
    print (su,mat0.files, 'TPS shape: ', [pow.shape for pow in pow_list])

    #=========================== Create dict for all results =================================    
    #if not path.exists(name_classif.format(su,freq)):
    kwargs = {}
    kwargs['names'], kwargs['channels'] = new_labels[idx_rois], mat0['channel'][idx_rois]
    kwargs['xyz'] = mat0['xyz'][idx_rois]

    # =========================== Select Power for 1 elec 1 freq =================================                 
    permut,auc = np.zeros((nelecs,nperm)),np.zeros((nelecs))
    for elec_num in range(nelecs):
        print('--» processing',su, 'elec', elec_num,'/',nelecs)
        pow_data_elec = [power[elec_num][:,np.newaxis] for power in pow_list]

        # create a data matrix, concatenate along the trial dimension
        x = np.concatenate(pow_data_elec, axis=0)
        print ('Size of the concatenated data: ', x.shape)
        y = np.hstack([np.array([i]*len(power)) for i, power in enumerate(pow_data_elec)])
        print ('Size of label for classif: ', len(y))

        clf = LDA()
        cv = SKFold(5)
        score, permutation_scores, pvalue = permutation_test_score(
        clf, x, y, scoring="roc_auc", cv=cv, n_permutations=nperm, n_jobs=-1)
        permut[elec_num] += permutation_scores
        auc[elec_num] += score
        print("Classification score %s (pvalue : %s)" % (score, pvalue))

    kwargs['perm'], kwargs['auc'] = permut, auc
    #Save plots
    np.savez(name_classif.format(su,freq), **kwargs)
    del x, y, pow_data_elec, permutation_scores, score
    #else:
    #    print(name_classif.format(su,freq), 'already computed')
    del pow_list

-> Olfacto loaded
['tps_0', 'tps_1', 'label', 'channel', 'xyz']
PIRJ ['tps_0', 'tps_1', 'label', 'channel', 'xyz'] TPS shape:  [(18, 45), (18, 66)]
--» processing PIRJ elec 0 / 18
Size of the concatenated data:  (111, 1)
Size of label for classif:  111
Classification score 0.4774114774114774 (pvalue : 0.5684315684315684)
--» processing PIRJ elec 1 / 18
Size of the concatenated data:  (111, 1)
Size of label for classif:  111
Classification score 0.35286935286935284 (pvalue : 0.977022977022977)
--» processing PIRJ elec 2 / 18
Size of the concatenated data:  (111, 1)
Size of label for classif:  111
Classification score 0.5576312576312576 (pvalue : 0.2897102897102897)
--» processing PIRJ elec 3 / 18
Size of the concatenated data:  (111, 1)
Size of label for classif:  111
Classification score 0.38351648351648354 (pvalue : 0.942057942057942)
--» processing PIRJ elec 4 / 18
Size of the concatenated data:  (111, 1)
Size of label for classif:  111
Classification score 0.5284493284493286 (pvalue

Classification score 0.4510197132616488 (pvalue : 0.7202797202797203)
--» processing FERJ elec 28 / 32
Size of the concatenated data:  (231, 1)
Size of label for classif:  231
Classification score 0.5497114695340501 (pvalue : 0.2177822177822178)
--» processing FERJ elec 29 / 32
Size of the concatenated data:  (231, 1)
Size of label for classif:  231
Classification score 0.6358010752688171 (pvalue : 0.000999000999000999)
--» processing FERJ elec 30 / 32
Size of the concatenated data:  (231, 1)
Size of label for classif:  231
Classification score 0.5567813620071684 (pvalue : 0.16683316683316685)
--» processing FERJ elec 31 / 32
Size of the concatenated data:  (231, 1)
Size of label for classif:  231
Classification score 0.4448189964157706 (pvalue : 0.7672327672327672)
['tps_0', 'tps_1', 'label', 'channel', 'xyz']
CHAF ['tps_0', 'tps_1', 'label', 'channel', 'xyz'] TPS shape:  [(61, 21), (61, 55)]
--» processing CHAF elec 0 / 61
Size of the concatenated data:  (76, 1)
Size of label for cla

Classification score 0.3881818181818182 (pvalue : 0.8081918081918081)
--» processing CHAF elec 43 / 61
Size of the concatenated data:  (76, 1)
Size of label for classif:  76
Classification score 0.5854545454545456 (pvalue : 0.23276723276723277)
--» processing CHAF elec 44 / 61
Size of the concatenated data:  (76, 1)
Size of label for classif:  76
Classification score 0.3327272727272727 (pvalue : 0.936063936063936)
--» processing CHAF elec 45 / 61
Size of the concatenated data:  (76, 1)
Size of label for classif:  76
Classification score 0.52 (pvalue : 0.5214785214785215)
--» processing CHAF elec 46 / 61
Size of the concatenated data:  (76, 1)
Size of label for classif:  76
Classification score 0.5536363636363637 (pvalue : 0.3926073926073926)
--» processing CHAF elec 47 / 61
Size of the concatenated data:  (76, 1)
Size of label for classif:  76
Classification score 0.5854545454545454 (pvalue : 0.26573426573426573)
--» processing CHAF elec 48 / 61
Size of the concatenated data:  (76, 1)


Classification score 0.6028869047619048 (pvalue : 0.1028971028971029)
--» processing VACJ elec 1 / 39
Size of the concatenated data:  (114, 1)
Size of label for classif:  114
Classification score 0.32755952380952386 (pvalue : 0.991008991008991)
--» processing VACJ elec 2 / 39
Size of the concatenated data:  (114, 1)
Size of label for classif:  114
Classification score 0.577217261904762 (pvalue : 0.20679320679320679)
--» processing VACJ elec 3 / 39
Size of the concatenated data:  (114, 1)
Size of label for classif:  114
Classification score 0.39244047619047623 (pvalue : 0.8841158841158842)
--» processing VACJ elec 4 / 39
Size of the concatenated data:  (114, 1)
Size of label for classif:  114
Classification score 0.6302380952380953 (pvalue : 0.03896103896103896)
--» processing VACJ elec 5 / 39
Size of the concatenated data:  (114, 1)
Size of label for classif:  114
Classification score 0.5986904761904762 (pvalue : 0.0959040959040959)
--» processing VACJ elec 6 / 39
Size of the concatena

Classification score 0.4766314543483235 (pvalue : 0.6423576423576424)
--» processing SEMC elec 8 / 53
Size of the concatenated data:  (384, 1)
Size of label for classif:  384
Classification score 0.5693879621616782 (pvalue : 0.029970029970029972)
--» processing SEMC elec 9 / 53
Size of the concatenated data:  (384, 1)
Size of label for classif:  384
Classification score 0.5583608367401748 (pvalue : 0.052947052947052944)
--» processing SEMC elec 10 / 53
Size of the concatenated data:  (384, 1)
Size of label for classif:  384
Classification score 0.4250325763679588 (pvalue : 0.964035964035964)
--» processing SEMC elec 11 / 53
Size of the concatenated data:  (384, 1)
Size of label for classif:  384
Classification score 0.579887499626988 (pvalue : 0.011988011988011988)
--» processing SEMC elec 12 / 53
Size of the concatenated data:  (384, 1)
Size of label for classif:  384
Classification score 0.5241365521768973 (pvalue : 0.37462537462537465)
--» processing SEMC elec 13 / 53
Size of the co